
# Task 1 – Daten-Preprocessing

Ziel: Rohdaten aus `buildings.csv` explorieren, bereinigen, normalisieren und für weitere Schritte (Anreicherung / Visualisierung) vorbereiten.

Hinweis: Bei der Verwendung der [Wikimedia PAWS Instanz](https://hub-paws.wmcloud.org/hub), bitte dieses Notebook und die zugehörigen Dateien extra hochladen (mit dem kleinen Uploadpfeil in der Seitenleiste). (Github Link)[https://github.com/kristbaum/historical-data-hacking] zum Download der Zip-Datei.

## Installation von Abhängigkeiten

* [Pandas](https://pandas.pydata.org/) 
* [Numpy](https://numpy.org/)
* [Shapely](https://shapely.readthedocs.io/en/stable/)

In [ ]:
# Install required packages into this notebook's Python kernel
%pip install -q --upgrade pip
%pip install -q pandas numpy shapely  # shapely for geospatial stuff

In [ ]:
import pandas as pd
import numpy as np

# Viele Spalten enthalten gemischte / freie Werte
# → zunächst als Strings importieren vermeidez fehlerhafte automatische Typkonvertierungen.
buildings = pd.read_csv('../buildings.csv', dtype=str)
print(buildings.shape)

# Zeigt einen Ausschnitt der ersten 5 Zeilen der Tabelle
buildings.head()


## 1. Erste Profilierung

1. Zähle Null/Leer-Werte pro Spalte (`''`, `' '`, `NaN`).
2. Ermittle Anteil gefüllter Werte für Kernfelder: `ID`, `appellation`, `verbaleDating`, `locationLat`, `locationLng`.
3. Erzeuge eine Kurztabelle (DataFrame) mit (Spaltenname, Non-Null %, Anzahl unterschiedlicher Werte, Beispielwerte).

In [ ]:
# Kopiere Rohdata für Profilierung
raw_data = buildings.copy()

# Ersetze Leerstrings und Spaces durch NA (fehlende Werte)
data_with_normalized_nulls = raw_data.replace('', np.nan, inplace=True)

# Liste für Spaltenprofile erstellen
profile_list = []

# Iteriere über alle Spalten und sammle Statistiken
for column_name in data_with_normalized_nulls.columns:
    column_series = data_with_normalized_nulls[column_name]
    
    # Berechne Anteil nicht-Null Werte (in Prozent)
    non_null_percentage = column_series.notna().mean() * 100
    
    # Zähle einzigartige Werte (ohne NA)
    unique_value_count = column_series.nunique(dropna=True)
    
    # Sammle erste 3 Beispielwerte
    sample_values_str = ', '.join(column_series.dropna().unique()[:3])
    
    # Füge Profil zur Liste hinzu
    profile_list.append({
        'column': column_name,
        'non_null_pct': non_null_percentage,
        'n_unique': unique_value_count,
        'sample_values': sample_values_str
    })

# Erstelle DataFrame aus den Profilen und sortiere nach Anteil nicht-Null Werte
profile_df = pd.DataFrame(profile_list).sort_values('non_null_pct')
profile_df.head(15)


In [ ]:
# 1b) Entferne Spalten mit sehr wenigen/nicht vorhandenen Werten
# Schwelle in Prozent (anpassbar): Spalten mit weniger als diesem Anteil an nicht-leeren Werten werden entfernt
min_non_null_pct = 5.0  # z. B. 5%

# Behandle '' und ' ' als fehlend für die Berechnung (ohne Side-Effects auf buildings)
normalized = buildings.replace({'': pd.NA, ' ': pd.NA}).infer_objects(copy=False)
non_null_pct = normalized.notna().mean() * 100

# Optional: wichtige Spalten schützen, damit sie nicht versehentlich entfernt werden
protected_cols = [c for c in ['ID', 'appellation', 'verbaleDating', 'locationLat', 'locationLng'] if c in buildings.columns]

# Bestimme zu entfernende Spalten
_to_drop = non_null_pct[non_null_pct < min_non_null_pct].index.tolist()
# geschützte Spalten nicht droppen
_to_drop = [c for c in _to_drop if c not in protected_cols]

# Entferne Spalten in-place
before_cols = buildings.shape[1]
buildings.drop(columns=_to_drop, inplace=True, errors='ignore')
after_cols = buildings.shape[1]

print(f"Entfernte Spalten (non-null < {min_non_null_pct}%): {len(_to_drop)} | Vorher: {before_cols}, Nachher: {after_cols}")
if _to_drop:
    print('Beispiele:', ', '.join(_to_drop[:10]) + (' ...' if len(_to_drop) > 10 else ''))

# Optional: zeige die 10 leersten verbleibenden Spalten zur Kontrolle
print('\nLeerste verbleibende Spalten (Top 10):')
print(non_null_pct.loc[buildings.columns].sort_values().head(10))

Fragen:

- Welche Spalten sind fast leer und sollten ggf. ausgelagert / ignoriert werden?
- Gibt es offensichtliche Duplikate bei `ID`?

## 2. Bereinigung `verbaleDating`

Spalte enthält verbale Zeitangaben / Bereiche wie: `"1000-2020, 1773-1776"`.

Aufgaben:

1. Zerlege `verbaleDating` in einzelne Segmente (Trennzeichen: Komma) → normalisiere Leerzeichen.
2. Identifiziere Bereiche (Pattern `YYYY-YYYY`) vs. Einzeljahre (`YYYY`).
3. Baue eine normalisierte Tabelle `building_dates`: (`building_id`, `date_raw`, `year_start`, `year_end`, `is_range`, `precision`).
4. Berechne je Gebäude: minimaler Start, maximaler Endwert → `chronology_min`, `chronology_max` und füge sie wieder dem Haupt-DataFrame hinzu.
5. Detektiere Ausreißer (z. B. Jahr < 1000 oder > aktuelles Jahr). Markiere sie für manuelle Prüfung.

In [ ]:
import re
rows = []
for _, r in buildings[['ID','verbaleDating']].fillna('').iterrows():
    parts = [p.strip() for p in r.verbaleDating.split(',') if p.strip()]
    for p in parts:
        m_range = re.fullmatch(r'(\d{4})-(\d{4})', p)
        m_year  = re.fullmatch(r'(\d{4})', p)
        if m_range:
            y1, y2 = map(int, m_range.groups())
            rows.append((r.ID, p, y1, y2, True, 'year-range'))
        elif m_year:
            y = int(m_year.group(1))
            rows.append((r.ID, p, y, y, False, 'year'))
        else:
            # Nicht-parsbare Fälle separat behalten
            rows.append((r.ID, p, None, None, None, 'unparsed'))

building_dates = pd.DataFrame(rows, columns=['building_id','date_raw','year_start','year_end','is_range','precision'])


Optionale Normalisierung unparsed Tokens: RegEx erweitern (z. B. `ca. 1750`, `17. Jh.` → Mapping Tabellen). Dokumentiere Annahmen!

Metriken:

- Anteil parsebarer Segmente.
- Häufigste unparsed Muster (Top 10).

## 3. Geodaten-Validierung+Auswertung

1. Konvertiere `locationLat` / `locationLng` zu Float; markiere Zeilen, bei denen das misslingt.
2. Prüfe Wertebereiche (Lat ∈ [-90, 90], Lng ∈ [-180, 180]). 
3. a) Auf Hessen beschränken mit Geokoordinate ( eines gedachten Rechtecks um Hessen (einfach von einer Karte von Hand geschätzt))
3. b) Auf das historische Hessen Nassau beschränken mit Geoshape. Daten von [OpenHistoricalMap](https://www.openhistoricalmap.org/relation/2690442) und dann [Abfrage mittels overpass](https://overpass-turbo.openhistoricalmap.org/?Q=%5Bout%3Ajson%5D%5Btimeout%3A120%5D%3B%0Arel%282690442%29%3B%0A%28._%3B%3E%3B%29%3B%0Aout%20body%20geom%3B&C=50.004209%3B11.518449%3B7) (Export Button, geoshape wählen)

In [ ]:
def to_float(s):
    try:
        return float(s)
    except (TypeError, ValueError):
        return pd.NA

coords = buildings[['locationLat','locationLng']].map(to_float)
valid_lat = coords.locationLat.between(-90,90)
valid_lng = coords.locationLng.between(-180,180)
buildings['coord_valid'] = valid_lat & valid_lng

In [ ]:
# 3a) Filter: Bounding Box für Hessen (vereinfachte Methode)
# Bounding Box ungefähr für Hessen (Nordwest / Südost):
NW_LAT, NW_LNG = 51.599, 8.072   # Nordwestlichster Punkt
SE_LAT, SE_LNG = 49.495, 10.088  # Südöstlichster Punkt


# Min/Max für Bereichsprüfung bestimmen
min_lat, max_lat = SE_LAT, NW_LAT
min_lng, max_lng = NW_LNG, SE_LNG

# Masken für BBox prüfen (nur valide Koordinaten berücksichtigen)
in_bbox = (
    coords.locationLat.between(min_lat, max_lat)
    & coords.locationLng.between(min_lng, max_lng)
)
mask_bbox = in_bbox & buildings['coord_valid'].fillna(False)

# Ergebnis: Gefilterte Entitäten in Hessen (via BBox)
buildings['in_hesse_bbox'] = mask_bbox
buildings_hesse_bbox = buildings.loc[mask_bbox].copy()

print(
    f"BBox Hessen: lat [{min_lat}, {max_lat}], lng [{min_lng}, {max_lng}] | "
    f"Treffer: {mask_bbox.sum()} von {len(buildings)}"
)
buildings_hesse_bbox.head(10)

In [ ]:
# 3b) Punkt-in-Polygon: Historisches Hessen-Nassau
import json
from shapely.geometry import shape, Point

# GeoJSON laden (Datei im aktuellen Ordner erwartet)
with open('./hessen_nassau.geojson', 'r', encoding='utf-8') as f:
    gj = json.load(f)

# Geometrie aus FeatureCollection/Feature/Geometry entnehmen
geom = (
    gj['features'][0]['geometry'] if gj.get('type') == 'FeatureCollection'
    else (gj['geometry'] if gj.get('type') == 'Feature' else gj)
)
hesse_nassau_polygon = shape(geom)

# Nur auf Zeilen mit gültigen Koordinaten prüfen
valid_mask = buildings['coord_valid'].fillna(False)
inside_series = pd.Series(False, index=buildings.index)
inside_series.loc[valid_mask] = coords.loc[valid_mask].apply(
    lambda r: hesse_nassau_polygon.contains(Point(float(r['locationLng']), float(r['locationLat']))),
    axis=1
)

buildings['in_hesse_nassau_geo'] = inside_series
print(f"Geo-Shape (Hessen-Nassau): Treffer {buildings['in_hesse_nassau_geo'].sum()} von {len(buildings)}")
buildings.loc[buildings['in_hesse_nassau_geo']].head(10)

## 4. Normalisierung mehrfacher Rollen-/Personenfelder

Viele Spalten enden auf Sequenzen `_1`…`_10` (z. B. `ARCHITECTS_1`, `ARCHITECTS_2`, ...). Werteform: `uuid|Nachname, Vorname`.

Ziel: Long-Format-Relation `building_person_roles`:
(`building_id`, `role` (z. B. `ARCHITECTS`), `sequence` (Nummer), `person_id` (UUID), `person_label`).

Aufgaben:

1. Identifiziere alle Basis-Rollen (Teil vor letztem `_` + Ziffern).
2. Iteriere über alle diese Spalten, extrahiere Werte ≠ leer.
3. Teile an erster Pipe `|` → `person_id`, zweiter Teil `person_label` (Fallback: kompletter String falls kein `|`).
4. Entferne potenzielle Dubletten (gleiche Kombination building_id + role + person_id).
5. Erzeuge optionale Personentabelle `persons` (distinct `person_id`, `person_label`, `label_clean`).
6. Bereinige `person_label`: Whitespace trimmen, vereinheitliche Kommaspacing.

In [ ]:
role_cols = [c for c in buildings.columns if re.search(r'_\d+$', c)]
base_roles = sorted(set(re.sub(r'_\d+$','', c) for c in role_cols))
rows = []
for role in base_roles:
    for i in range(1, 11):
        col = f'{role}_{i}'
        if col not in buildings.columns: 
            continue
        for _, r in buildings[['ID', col]].iterrows():
            val = r[col]
            if pd.isna(val) or not str(val).strip():
                continue
            if '|' in val:
                pid, label = val.split('|',1)
            else:
                pid, label = None, val
            rows.append((r.ID, role, i, pid, label.strip()))

building_person_roles = pd.DataFrame(rows, columns=['building_id','role','sequence','person_id','person_label'])
persons = (building_person_roles
           .dropna(subset=['person_label'])
           .groupby(['person_id','person_label'], dropna=False)
           .size().reset_index(name='count'))

Validierung:

- Wie viele Rollen wurden extrahiert?
- Top 3 Personen je Rolle.

## 5. Qualitätsmetriken & Checks

Erzeuge kleine Metriken (als DataFrame oder Markdown):

- Parsebarkeit Datum (%).
- Anzahl unparsed Datumseinträge.
- Anzahl extrahierter Personenbeziehungen.
- #Distinct Personen.
- Anteil gültiger Koordinaten.

## OpenRefine

Ziel: Schnelle, reproduzierbare Bereinigung & Normalisierung zentraler Felder aus `buildings.csv` mittels OpenRefine.

> Fokus: Sicht auf typische OpenRefine-Operationen (Facets, Clustering, Transformations, Splits, Rekonsilierungsvorbereitung). Dauer: ca. 30–40 Minuten.

### 1. Import

- Lade `buildings.csv` in OpenRefine.
- Alle Spalten zunächst als Text lassen (kein Auto-Parsing von Zahlen/Datumsangaben aktivieren).
- Projektname: `buildings_raw`.

### 2. Grundlegende Sichtbarkeit

- Entferne (nur in der Ansicht, nicht dauerhaft) extrem leere Spalten via Facet → „Facet by blank“ und spätere Auswahl für Export.
- Erzeuge eine Text-Facet auf `appellation` → erkenne Varianten / Dubletten.

### 3. Bereinigung `verbaleDating`

Ziel: Segmentierung & Vor-Normalisierung für spätere Python-Verarbeitung.

Schritte:

1. Expression (GREL) Trim: `value.trim()` (Spalte bearbeiten → Zellen transformieren).
2. Ersetze mehrere Leerzeichen: `value.replace(/\s+/,' ')`.
3. Split bei Komma (Spaltenmenü → „Edit cells → Split multi-valued cells“ → Separator `,`).
4. Neue Spalte aus dieser (JSON-Serialisierung einzelner Werte für Kontrolle): `value` (Beibehalten). Optional: Werte mit Regex-Facet `^[0-9]{3,4}(-[0-9]{3,4})?$` filtern (parsbar vs. unparsed).
5. Erzeuge Booleanspalte `is_range`: GREL: `value.match(/^[0-9]{3,4}-[0-9]{3,4}$/) != null`.
6. Erzeuge Spalten `year_start` und `year_end`:
   - Falls Range: `value.match(/^(\d{3,4})-(\d{3,4})$/)[0]` & `[1]`
   - Falls Einzeljahr: `value` in beide kopieren.
7. Filtere Ausreißer: Facet `year_start` → Numeric Facet → Werte außerhalb 800–(aktuelles Jahr) markieren.

Hinweis: Exportiere Zwischenergebnis als `building_dates_openrefine.csv` für Abgleich mit Pandas-Parsing.

### 4. Personen-/Rollenfelder (Beispiel: ARCHITECTS)

Ziel: Long-Format Grundlage.

Schritte:

1. Wähle Spalten `ARCHITECTS_1` … `ARCHITECTS_10`.
2. „Edit columns → Join columns“ (Separator `||`), erzeuge Sammelspalte `ARCHITECTS_JOIN`.
3. Split multi-valued cells an `||` → leere entfernen.
4. Entferne Duplikate (Facet by text, Auswahl blank → ausschließen).
5. Extrahiere UUID & Label:
   - Neue Spalte `person_id`: GREL: `if(value.contains('|'), value.split('|')[0], null)`
   - Neue Spalte `person_label_raw`: GREL: `if(value.contains('|'), value.split('|')[1], value)`
6. Clean Label: Neue Spalte `person_label`: `person_label_raw.trim().replace(/\s*,\s*/, ', ')`.
7. Cluster (Edit cells → Cluster & edit) auf `person_label` (Method: key collision + metaphone3). Prüfe Zusammenführungen.
8. Export als `architects_roles.csv` (Spalten: `building_row_index` / `person_id` / `person_label`). Optional: füge Quellspalte `ARCHITECTS` hinzu.

### 5. Vorbereitung für Wikidata-Reconciliation

Ziel: Spalte (z. B. `appellation` oder aufbereitete Personennamen) für spätere Abgleichung.

Schritte:

1. Entferne offensichtliche Zusätze (Regex Replace): Beispiel: `value.replace(/,\s*Deutschland$/,'')` (nur wenn sinnvoll!).
2. Normalisiere Großschreibung: `value.toTitlecase()` (sparsam einsetzen, um historische Schreibweisen nicht zu verfälschen).
3. Exportiere Liste eindeutiger Werte (`Facet → Export → tabular`) als `appellations_unique.csv`.

### 6. Audit & Undo/Redo

- Nutze das Undo/Redo Panel, dokumentiere die angewandten Schritte (Screenshot / JSON). Export: „Extract…“ → Speichere Transformations-JSON als `openrefine-history.json` für Reproduzierbarkeit.

### 7. Kurzer Qualitätsbericht (Markdown außerhalb OpenRefine)

- Anzahl ursprünglicher vs. bereinigter `verbaleDating` Tokens.
- Anzahl zusammengeführter Personenlabels durch Clustering.
- Wichtige offene Problemfälle (Stichpunkte).
